# Docker Command vs Entrypoint

### Introduction

So far we have used CMD when building our Docker images.  However, the `CMD` function can also be paired with `Entrypoint`.  In this lesson, we'll review the CMD function, and then move into `Entrypoint`.

### Reviewing Command

As we know, CMD is often the last line of our Dockerfile.

For example, below is our Dockerfile for booting up a flask application (currently located inside of `web_app/app`).

```Dockerfile
FROM python:3.8.5-alpine3.12

WORKDIR /app

EXPOSE 80
ENV FLASK_APP=app.py

COPY ./src .
RUN pip3 install -r requirements.txt

CMD ["flask", "run", "--host=0.0.0.0", "--port=80"]
```

Now why is CMD our last line?  Well when we build a Docker image, all of the previous lines are packaged up in our image.  That last command line is run as the last step, only when we finally run the docker image.

Let's see this.  From the `web_app/app` folder, we can run the following.

```bash
docker build -t web_app .
```

<img src="./docker_build.png">

So notice that when we build the image it stops right before the CMD line, with `RUN pip3 install -r requirements.txt`.  

It's only when we call `docker run web_app` that the CMD line is executed.

<img src="./cmd_fn.png" width="60%">

One feature of CMD is that it's really just specifying the default behavior at runtime (when we call docker run).  We can override this by providing a additional arguments after `docker run image_name`.  For example, below we'll provide the command for running flask on port 3000.

```bash
docker run web_app flask run --port=3000
```

Or we can not have flask run at all, and just provide the command for bashing into the container.

```bash
docker run -it web_app sh
```

### Moving to Entrypoint 

Ok, so features of the CMD function are that it's run when building an image.  And that we can override it, by specifying additional arguments after `docker run image_name`.



Now let's move to using `entrypoint`.  `Entrypoint` is similar to `CMD` in that it is only run at runtime, when calling `docker run image_name`.  And like `CMD`, there can only be one `Entrypoint` per `Dockerfile`.

However it is different from CMD in that it *cannot* be overriden.

Let's see this.  Update the Dockerfile to the following.

```bash
FROM python:3.8.5-alpine3.12

WORKDIR /app

EXPOSE 80
ENV FLASK_APP=app.py

COPY ./src .
RUN pip3 install -r requirements.txt

ENTRYPOINT ["flask", "run"]
```

And then rebuild the image.

`docker build -t web_app .`

And if you run it, everything operates the same.

```bash
docker run  web_app
```

A difference is if you try to provide arguments *after* running `docker run`.  For example, let's say when running the image's container, we want to change the port.  This is how we can do it.

```bash
docker run  web_app --port=80
```

So the `ENTRYPOINT` line is still run, and now we just pass an additional argument to our command with `--port=80`.  So when passing through the argument above, on runtime, we were running something like:

```
flask run --port=80
```

But remember, the `flask run` part is not overriden with our line `docker run web_app --port=80`.  We are just adding additional arguments to what's specified in our entrypoint.

We can see this a bit better if we try to sh into our container again.  Run the following.

```bash
docker run -it web_app sh
```

<img src="./docker-sh.png" width="60%">

So above we get an error.  By running `docker run -it web_app sh`, we are passing `sh` as an argument to what's specified after entrypoint.  So in this case, we are basically running `flask run sh`, which isn't a valid flask command.

* Entrypoint with CMD

Ok, now we can actually pair entrypoint with command.  Update your Dockerfile, so the last few lines look like the following:

```dockerfile
ENTRYPOINT ["flask", "run"]

CMD ["--port=80"]
```

Then rebuild the image.

```bash
docker build -t web_app .
```

And run the container.

```bash
docker run web_app
```

<img src="./port-80.png" width="60%">

This time if we run `docker run`, you can see that `--port=80` is passed as an argument to our `ENTRYPOINT` command.  So above, `--port=80` is passed to `flask run`.

And just like always, we can override what's specified in our `CMD` by adding arguments after `docker run image_name`.  For example, let's have our flask application run on `port=3000` instead of `80`.

```bash
docker run web_app --port=3000
```

<img src="./port-more.png" width="60%">

> So above, the `--port=3000` overrides the default argument specified in CMD, `CMD ["--port=80"]`, and that overriding argument is passed to what's specified in ENTRYPOINT, `ENTRYPOINT ["flask", "run"]`.  So with 

```bash
docker run web_app --port=3000
```

We are effectively running:

`flask run --port=3000`

### Summary

In this lesson, we reviewed the `CMD` function in a Dockerfile.  And saw that our CMD function is not packaged up in our Docker image, but instead is only run when calling `Docker run image_name`.  We also saw that what's specified in `CMD` is *default* behavior, and can be overridden by passing additional arguments to `Docker run image_name`.

For example, if we have a CMD of `CMD ["flask", "run", "--port=80"]`, we can override it with a flask commmand to run on port 3000 with:

`docker run web_app flask run --port=3000`

Entrypoint is similar in that it is also only run when container is being run, however we cannot override the Entrypoint command.  We can only pass additional arguments to it.  For example, if we have the following in our Dockerfile:

```
Entrypoint ["flask", "run"]
```

Then `flask run` must be run everytime we run our container, but we can pass additional arguments to it with `docker run web_app --port=80`.  

> Notice that when we have a CMD function, here we are only passing through the additional arguments (`--port=80`), and we are not re-specifying the `flask run` part like we did with command.  With entrypoint that command of `flask run` stays in tact.

Finally, if we have both `entrypoint` and `CMD` specified in our Dockerfile, then what's passed into CMD is a passed into our entrypoint command as a default argument.

```dockerfile
ENTRYPOINT ["flask", "run"]

CMD ["--port=80"]
```

But this default argument can be overridden when booting up the container.

```bash
docker run web_app --port=3000
```

<img src="./port-more.png" width="60%">